## Objective

> 모든 언어에서의 핵심은 바로 변수를 어떤 식으로 처리하는 가에 달려 있다. Python은 다른 언어와 좀 이질적인 방식으로 변수를 처리한다. 그것에 대해 알아보자.

### 1. 첫번째 진실 : 파이썬은 Type이 없는 함수

In [1]:
x = 1

In [2]:
type(x)

int

> 우리는 흔히, 파이썬에서는 int, str, float 같은 기본 타입이 있다고 착각한다. 사실 이것들은 타입이 아니라, Class이다. 왜 이렇게 말할 수 있냐면 `type`이었다면 내부 메소드는 없어야 한다. 왜냐하면, 보통 자료형은 just only data이지, 내부 메소드가 없기 때문이다.

In [3]:
# little endian 형식으로 바이트 표기
x.to_bytes(1,'little')

b'\x01'

> 파이썬은 기본형식인 int에 마저 위와 같이 내부 메소드가 존재한다. 

그렇다면, type이 없다면 `type()`에서 보여주는 결과는 무엇인가? 어떤 클래스인지 지칭하는 메소드일 뿐이다.

In [4]:
class Foo(object):
    def __init__(self,name):
        self.name = name

In [5]:
y = Foo("hello")

In [6]:
type(y)

__main__.Foo

그렇기 때문에 우리는 심지어 `int`에다가 저 `Foo` class를 넣을수도 있다.

In [7]:
int = Foo

In [8]:
x = 5

In [9]:
z = Foo("Hello")

In [10]:
type(z)

__main__.Foo

> 그럼 우리의 int는 어디로 사라졌을까. 기본적으로 int는 builtin된 클래스이다. 그렇기 때문에 `__builtin__`에 존재한다. 

In [11]:
int = __builtin__.int

In [12]:
int(10)

10

In [13]:
# 그럼 이걸 해보면 어떨까? 커널이 바로 죽는다 조심하라
# __builtin__.int = Foo

> 파이썬에서는 모든 것을 객체(object)이다. 파이썬은 모든 것들을 객체로 처리하고, 객체로 반환한다. 

### 두번째 진실 : 파이썬은 가끔 포인터를 기괴하게 처리한다. 

In [61]:
x = 1

> x의 메모리 위치는 어디에 있을까? 이럴 때, 우리는 `id`를 이용한다.

In [62]:
id(x)

4513092960

In [63]:
x = x + 1

> x에 값을 하나 올렸다. 그럼 위치가 어디에 있을까? 

In [64]:
id(x)

4513092992

> 위치가 바뀌었다.

> 이는 int라는 데이터 타입이 immutable datatype이기 때문이다. immutable datatype의 특징은 해당 주소값에 값이 바뀌지 않는다. 

#### 주요한 것들은 아래와 같다
![](https://cdn-images-1.medium.com/max/1600/1*uFlTNY4W3czywyU18zxl8w.png)

### 세 번째 진실 : 파이썬에도 Call-by-Value와 Call-by-Reference가 존재한다!

In [67]:
def add_1(x):
    return x + 1

x = 1
print("before : ", x)
add_1(x)
print("after : ", x)

before :  1
after :  1


> before과 after는 동일하다!

In [68]:
def append_1(x):
    return x.append(1)

x = [1]
print("before : ", x)
append_1(x)
print("after : ", x)

before :  [1]
after :  [1, 1]


> 다르다..! 이것이 핵심 문제. Immutable의 값들은 call-by-value로 처리하고, mutable의 값들은 call-by-reference로 처리한다. immutable은 몇개 없다. 즉 대부분 객체는 mutable이고, 우리가 다루는 객체들도 기본적으로 mutable이다. 그러므로 저런 문제가 발생할 수 있으니 늘 주의해야 한다. 그렇다면 위와 같은 케이스를 방지하려면 어떻게 해야 할까?

In [69]:
import copy

In [71]:
def append_1(x):
    y = copy.deepcopy(x)
    return y.append(1)

x = [1]
print("before : ", x)
append_1(x)
print("after : ", x)

before :  [1]
after :  [1]


> 위와 같이 명시적으로 객체를 만들어서 처리해야 한다. 그래서 디버깅할 때, 이러한 이슈가 없는지 사전에 확인해보아야 한다.

## 위의 개념으로 우리가 설명할 수 있는 현상들

1) list concatenation VS list extend

In [73]:
%%timeit 
x = []
y = [1,2,3,4,5]
for _ in range(100):
    x.extend(y)

9.66 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [74]:
%%timeit 
x = []
y = [1,2,3,4,5]
for _ in range(100):
    x = x + y

52.7 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


> 두 가지 연산은 정확히 같은 결과를 내뱉지만, 시간은 꽤나 차이가 난다. 왜일까

In [76]:
x = []
print("before : ", id(x))
y = [1,2,3,4,5]
x.extend(y)
print("after : ", id(x))

before :  4550536904
after :  4550536904


> x의 위치가 일정하다. 즉 call-by-reference로 동작한다.

In [77]:
x = []
print("before : ", id(x))
y = [1,2,3,4,5]
x = x + y
print("after : ", id(x))

before :  4550536968
after :  4547931912


> x의 위치가 다르다. 즉 call-by-value로 동작한다. 이는 내부에서 명시적으로 값을 복사하는 불필요한 행위가 추가되기 때문이다. 그래서 불필요한 상황에서 list concatenation을 반복적으로 사용하는 것은 피해야 한다. 

2) numpy flatten vs ravel

In [115]:
import numpy as np

In [116]:
x = np.zeros([100,100])

In [117]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

> numpy에는 이것을 직렬로 1D으로 만들어주는 연산자가 두개가 있다.

In [118]:
x.flatten()

array([0., 0., 0., ..., 0., 0., 0.])

In [119]:
x.ravel()

array([0., 0., 0., ..., 0., 0., 0.])

> 이 두개의 차이는 무엇일까?

In [124]:
x = np.zeros([100,100])
y = x.flatten()
y[0] = 1
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

> y와 x는 다른 객체이다.

In [125]:
x = np.zeros([100,100])
y = x.ravel()
y[0] = 1
x

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

> y와 x는 같은 객체이다. 